In [4]:
import pandas as pd
import csv
from config import ROOT_DIR
import sys

In [10]:
csv.field_size_limit(sys.maxsize)

with open(ROOT_DIR / ".data" / "scrapped_articles.csv", 'r', encoding='utf-8', errors='replace') as infile, \
     open(ROOT_DIR / ".data" / "scrapped_articles_clean.csv", 'w', encoding='utf-8') as outfile:
    reader = csv.reader(infile)
    writer = csv.writer(outfile)
    for row in reader:
        try:
            writer.writerow(row)
        except Exception:  # log and skip
            print(f"Error writing row: {row}")
            continue

scrapped_df = pd.read_csv(ROOT_DIR / ".data" / "scrapped_articles_clean.csv")

In [11]:
scrapped_df

,website,title,url,content,domain,slop
0,NY Times,Submit a Crossword,https://www.nytimes.com/article/submit-crosswo...,"<html lang=""en"" class=""story nytapp-vi-article...",News,Non-Slop
1,NY Times,Part 3The Best-Laid Plans,https://www.nytimes.com/interactive/2025/03/29...,"<html lang=""en"" class=""nytapp-vi-interactive n...",News,Non-Slop
2,NY Times,"Part 4Breaches of Trust, and of Borders",https://www.nytimes.com/interactive/2025/03/29...,"<html lang=""en"" class=""nytapp-vi-interactive n...",News,Non-Slop
3,NY Times,Jack Dura/Associated Press,https://www.nytimes.com/2025/03/31/climate/gre...,"<html lang=""en"" class=""story nytapp-vi-article...",News,Non-Slop
4,NY Times,"Part 2‘When You Defeat Russia, We Will Make Yo...",https://www.nytimes.com/interactive/2025/03/29...,"<html lang=""en"" class=""nytapp-vi-interactive n...",News,Non-Slop
...,...,...,...,...,...,...
1268,Listverse,Animals,https://listverse.com/science-nature/animals/,"<html class=""no-js "" lang=""en-US""><!--<![endif...",Lifestyle,Slop
1269,Listverse,The Arts,https://listverse.com/entertainment/the-arts/,"<html class=""no-js "" lang=""en-US""><!--<![endif...",Lifestyle,Slop
1270,Listverse,Health,https://listverse.com/leisure-travel/health/,"<html class=""no-js "" lang=""en-US""><!--<![endif...",Lifestyle,Slop
1271,Listverse,Crime,https://listverse.com/people-politics/crime/,"<html class=""no-js "" lang=""en-US""><!--<![endif...",Lifestyle,Slop
